## Reconhecimento de dígitos com MLP

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import dataloader
from torchvision.transforms import ToTensor

In [2]:
data_train = datasets.MNIST(root='DataSets', 
                            train=True,
                            download=True,
                            transform=ToTensor()) # Normaliza os dados
data_test = datasets.MNIST(root='DataSets', 
                            train=False,
                            download=True,
                            transform=ToTensor())

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128
epochs = 20

In [4]:
train_loader = dataloader.DataLoader(dataset=data_train, batch_size=128, shuffle=True)
test_loader = dataloader.DataLoader(dataset=data_test, batch_size=128, shuffle=False)


In [5]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size, output_size):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.tanh1 = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.tanh2 = nn.Tanh()
        self.fc3 = nn.Linear(hidden_size,output_size)
    
    def forward(self, x):
        # Achatando a imagem de [batch_size, 1, 28, 28] para [batch_size, 784]
        x = x.view(-1, 28*28)
        out = self.fc1(x)
        out = self.tanh1(out)
        out = self.fc2(out)
        out = self.tanh2(out)
        out = self.fc3(out)
        # Nenhuma função de ativação aqui, pois CrossEntropyLoss cuidará disso
        return out

mod = MLP(28*28, 256, 10)
print("Arquitetura do Modelo:")
print(mod)

Arquitetura do Modelo:
MLP(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (tanh1): Tanh()
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (tanh2): Tanh()
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [6]:
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=mod.parameters(), lr=0.001)

In [ ]:
history_norm = {'loss': [], 'accuracy': []}
total_step = len(train_loader)

print("\n--- INICIANDO TREINAMENTO COM DADOS NORMALIZADOS ---")
mod.to(device)
mod.train()
for epoch in range(5):
    total_loss = 0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        images = images.to(device)
        labels = labels.to(device)
        outputs = mod(images)
        loss = loss_func(outputs, labels)

        # Backward and optimize
        opt.zero_grad()
        loss.backward()
        opt.step()

        # Monitoramento
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = total_loss / total_step
    epoch_acc = 100 * correct / total
    history_norm['loss'].append(epoch_loss)
    history_norm['accuracy'].append(epoch_acc)

    print(f'Época [{epoch+1}/{20}], Perda: {epoch_loss:.4f}, Acurácia: {epoch_acc:.2f}%')

print("--- TREINAMENTO CONCLUÍDO ---")


--- INICIANDO TREINAMENTO COM DADOS NORMALIZADOS ---


NameError: name 'model_norm' is not defined

In [ ]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # --- 3. MOVER OS DADOS DE TESTE PARA A GPU --- ### GPU ###
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Para a matriz de confusão, precisamos trazer os dados de volta para a CPU
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy_test = 100 * correct / total
print(f'\nAcurácia do modelo no conjunto de teste: {accuracy_test:.2f} %')
